In [7]:
from transformers import AutoTokenizer
from pymongo import MongoClient
from torch.utils.data import Dataset

In [8]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [3]:
class KGLMDataset(Dataset):
    def __init__(self, port, db, collection):
        self.client = MongoClient('localhost', port)
        self.db_name = db
        self.collection_name = collection
        self.collection = self.client[db][collection]
        self.length = self.client[self.db_name].command("collstats", self.collection_name)['count']

    def  __getitem__(self, idx):
        item = {}
        doc = self.collection.find_one({'_id': str(idx)})
        item["input"] = doc['verbalization']
        item["outputs"] = doc['target']
        return item
        
    def __len__(self):
        return self.length

In [18]:
tokenizer(['<extra_id_2>'])

{'input_ids': [[32097, 1]], 'attention_mask': [[1, 1]]}

In [19]:
# tokenizer.special_tokens_map

In [22]:
tokenizer.add_special_tokens({'sep_token': '[SEP]'})

1

In [23]:
train_dataset = KGLMDataset(27017, 'KGLM', 'train')

In [24]:
train_dataset[0]

{'input': 'predict [SEP] Lalit Kumar Goel instance of [SEP] ',
 'outputs': 'human'}

In [26]:
encoded = tokenizer(train_dataset[0]['input'])
encoded


{'input_ids': [9689, 32100, 325, 4250, 23400, 1263, 15, 40, 3421, 13, 32100, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
tokenizer.decode(encoded['input_ids'])

'predict[SEP] Lalit Kumar Goel instance of[SEP]</s>'